# Table of Contents

[Imports](#Imports)

[Playground](#Playground)

[Spotify Functions](#Spotify-Functions)

[Songkick Functions](#Songkick-Functions)

[Analysis Functions](#Analysis-Functions)

[Analysis](#Analysis)

# Imports

[Back to top](#Table-of-Contents)

In [2]:
#import necessary libraries
import requests
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import re
import spotipy
import base64
import teetool as tt
from collections import Counter
import libpysal
from pysal.model import spreg
from libpysal.weights import Queen, Rook, KNN, Kernel
import pysal
from pysal.explore import esda
from pysal.viz.splot import esda as esdaplot
import cenpy
import datetime
from datetime import date
from spotipy.oauth2 import SpotifyClientCredentials
from census import Census
import censusgeocode
import us
import folium
import statsmodels.formula.api as sm
import numpy
import statistics
import itertools as it
import math

# various API keys, necessary actions before beginning
api_key = "fCnCRQ1S6LDAS7zQ"
client_id = 'f5a0fe76389c42c0b85c832f1de66cf7'
client_secret = 'e9d4fc0874f345b2b636a6e307384bb7'
census_key = '684a12c6b65d2fe14bdf6d8354da8fe3aff08d70'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cen = Census(census_key)

# creation of a shapefile with both countries and US states included
world = gpd.read_file('Countries_WGS84/Countries_WGS84.shp') #open file
states_shp = gpd.read_file('states_21basic/states.shp') #open file

s2 = states_shp[['geometry', 'STATE_ABBR']]
s2.columns = ['geometry', 'name']

w2 = world[['geometry', 'CNTRY_NAME']]
w2.columns = ['geometry', 'name']

ws = w2.append(s2)

print(ws)


#maybe remove festivals - songkick
# maybe market - spotify

/Users/kalebtsegaye/miniconda3/envs/env1/lib/python3.7/site-packages/pysal/explore/segregation/network/network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
/Users/kalebtsegaye/miniconda3/envs/env1/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


                                             geometry                 name
0   POLYGON ((-69.8822326660156 12.4111099243165, ...                Aruba
1   (POLYGON ((-61.7388916015625 17.5405540466309,...  Antigua and Barbuda
2   POLYGON ((61.2765579223633 35.6072463989258, 6...          Afghanistan
3   POLYGON ((-5.15213489532459 30.1804695129396, ...              Algeria
4   (POLYGON ((45.0258293151855 41.0305480957031, ...           Azerbaijan
..                                                ...                  ...
46  POLYGON ((-94.46169128436209 34.19676535931296...                   AR
47  (POLYGON ((-93.70752428365972 30.2395783556275...                   LA
48  (POLYGON ((-80.7856622716253 28.78519435427305...                   FL
49  (POLYGON ((-88.49752727880752 48.1737953723300...                   MI
50  (POLYGON ((-161.3337853466416 58.7332483821643...                   AK

[302 rows x 2 columns]


# Playground

This was for testing certain functions.

[Back to top](#Table-of-Contents)

# Spotify Functions

[Back to top](#Table-of-Contents)

In [3]:
# spotify function definitions

# given a name/id or genre, will provide a list of 
# artists that exist in relation and also are within the follower limit

# custom function used to sort by num of shared genres
def custom_sort(x):
    return x[2]

query_count = 0

# handles counting the genres for related artists
def ra_help(artid, genres, followers, lessthan):
    global query_count # count of the amount of spotify queries made
    alist = []
    if query_count < 10:
        query_count += 1
        for a in sp.artist_related_artists(artid)['artists']: # get, loop through related artists
            gc = 0
            if (followers == -1): # no need to count followers if not specified
                for g in a['genres']:    
                    if g in genres:
                        gc += 1
                if gc > 0 and ([item for item in alist if item[1] == a['id']] == []):
                    alist.append((a['name'], a['id'], gc, a['followers']['total']))
            else: # count followers
                if (lessthan == True):
                    if (a['followers']['total'] < followers):
                        for g in a['genres']:    
                            if g in genres:
                                gc += 1
                        if gc > 0 and ([item for item in alist if item[1] == a['id']] == []):
                            alist.append((a['name'], a['id'], gc, a['followers']['total']))
                elif (lessthan == False):
                    if (a['followers']['total'] >= followers):
                        for g in a['genres']:    
                            if g in genres:
                                gc += 1
                        if gc > 0 and ([item for item in alist if item[1] == a['id']] == []):
                            alist.append((a['name'], a['id'], gc, a['followers']['total']))
                
        return sorted(alist, key = custom_sort, reverse = True) # sorts by amount matched genres, more = better
                    
# used to faciliate recursive calls - 
# allows me to find the related artists in each level of separation from the root
# before moving on to the root artist
def ra_rec(lst, genres, followers, lessthan):
    global query_count
    lst2 = []
    query_count += 1
    for a in lst:
        if query_count < 10:
            lst2+=(ra_help(a[1], genres, followers, lessthan))
    if query_count < 10:
        lst2 += (ra_rec((sorted(lst2, key = custom_sort)), genres, followers, lessthan))    
    
    return sorted(lst2, key = custom_sort, reverse = True)

# main function to be called
def related_artists(artname='', artid='', genre='', followers=-1, lessthan = True, llen = -1):
    if (artname == '' and artid == '' and genre == ''):
        print('please include name, id, or genre')
        return
    else:
        global query_count #counts spotify queries
        query_count = 0
        alist = []
        if (artname != ''):
            se = sp.search(q = artname, limit = 1, type = 'artist')
            query_count += 1
            print("related_artists search " + str(query_count))
            if (se['artists']['total'] > 0):
                artist = se['artists']['items'][0]
                artid = artist['id']
                genres = artist['genres']
        elif (artid != ''):
            artist = sp.artist(artid)
            genres = artist['genres']
            
        
        for a in sp.artist_related_artists(artid)['artists']:
            genres+=a['genres']
            

        lst = ra_help(artid, genres, followers, lessthan)
        query_count += 1
        print("related_artists ra_help " + str(query_count))
                
        if query_count < 10:
            lst+=(ra_rec(lst, genres, followers, lessthan))
            
    if llen > 0:
        return sorted(list(set(lst)), key = custom_sort, reverse = True)[0:llen]
    else:
        return sorted(list(set(lst)), key = custom_sort, reverse = True)


# Songkick Functions

[Back to top](#Table-of-Contents)

In [8]:
# function definitions

# takes json string from songkick for an artist and makes points based on the results
def make_points(json_str, artist):
    df = pd.DataFrame(
    {'Artist': [],
     'Venue': [],
     'City': [],
     'State/Province': [],
     'Country': [],
     'Longitude': [],
     'Latitude': []})

    i = 0
    
    if (json_str['resultsPage']['totalEntries'] != 0):
        for p in json_str['resultsPage']['results']['event']:
            #     print(p['popularity'])
            venue = p['venue']['displayName']
            lng = p['location']['lng']
            lat = p['location']['lat']
            city = p['location']['city']
            state = 'N/A'
            csplit = city.split(',')
            country = csplit[1]
        
            if (len(csplit) == 3): # handle Washington, DC
                if not ('Washington' == csplit[0] and 'DC' == csplit[1]):
                    city = csplit[0]
                    state = csplit[1]
                    country = csplit[2]
                else:
                    city = csplit[0] + ', ' + csplit[1]
                    state = 'N/A'
                    country = csplit[2].replace(' ', '')
        
            df.loc[i] = [artist] + [venue] + [city] + [state] + [country] + [lat] +[lng] #add to dataframe

            i+=1
            
        #convert to gdf to plot
        gdf = gpd.GeoDataFrame(
            df, geometry=[Point(x, y) for y, x in zip(df['Longitude'], df['Latitude'])])

        gdf2 = gdf
        gdf2 = gdf2.drop(list(range(1, len(gdf)))) # start point
        gdf3 = gdf
        gdf3 = gdf3.drop(list(range(0, len(gdf)-1))) # end point
    
        return(gdf, gdf2, gdf3)
    else:
        return -1
    
# finds the longest stretch of concerts for an artist, given a cutoff number of days, tr, that
# each show must play within. this is defaulted to 7.
def find_tour(json_str, artist, tr = 7, cutoff_yr = 2):
    df = pd.DataFrame(
    {'Artist': [],
     'Venue': [],
     'City': [],
     'State/Province': [],
     'Country': [],
     'Longitude': [],
     'Latitude': [],
     't': []})

    i = 0
    
    arr = []
    
    if (json_str['resultsPage']['totalEntries'] != 0):
        for p in json_str['resultsPage']['results']['event']:
            if 'US' in p['location']['city']:
                venue = p['venue']['displayName']
                lng = p['location']['lng']
                lat = p['location']['lat']
                city = p['location']['city']
                state = 'N/A'
                csplit = city.split(',')
                country = csplit[1]
        
                if (len(csplit) == 3): # handle Washington, DC
                    if not ('Washington' == csplit[0] and 'DC' == csplit[1]):
                        city = csplit[0]
                        state = csplit[1]
                        country = csplit[2]
                    else:
                        city = csplit[0] + ', ' + csplit[1]
                        state = city
                        country = csplit[2].replace(' ', '')
                    df.loc[i] = [artist] + [venue] + [city] + [state] + [country] + [lat] + [lng] + [p['start']['date']] # add to df
            
                i+=1
            
                if (len(p['location']['city'].split(',')) == 3): # making a list of dates to find longest tour
                    d = p['start']['date'].split('-')
                    arr.append(date(int(d[0]), int(d[1]), int(d[2])))
    else:
        return -1
    
    l = [0,0]
    first = 0
    last = 0
    
    #finding the indices of the longest tour
    for i in range(len(arr)-1):
        if (arr[i] != -1 and arr[i+1] != -1):
            if (((arr[i] - arr[i+1]).days > tr) or (i == len(arr)-2)):
                last = i
                if (last-first > l[1] - l[0]):
                    l = [first, last]
                first = i+1
        else:
            last = i
            if (last-first > l[1] - l[0]):
                l = [first, last]
            first = i+1
            
#     print(l)
    
    df2 = pd.DataFrame(
    {'Artist': [],
     'Venue': [],
     'City': [],
     'State/Province': [],
     'Country': [],
     'Longitude': [],
     'Latitude': [],
     't': []})
    
    # returning geodataframe of longest tour
    if (len(df) > 0):
        for i in range(l[0], l[1]+1):
            df2.loc[i] = df.loc[i]
        
        #convert to gdf to plot
        gdf = gpd.GeoDataFrame(
                df2, geometry=[Point(x, y) for y, x in zip(df2['Longitude'], df2['Latitude'])])

        return gdf
    else:
        return -1

# plots path of shows for artist
def run_plot(req, aname):
    gdfs = make_points(req.json(), aname)

    points_gdf = gdfs[0]
    print(points_gdf)
    start_gdf = gdfs[1]
    stop_gdf = gdfs[2]

    #current us basemap
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # fig, ax = plt.subplots(figsize=(15,15))
    ax = world[world.continent == 'North America'].plot(
#     ax = world.plot(
        color='white', edgecolor='black')

    ax.set_xlim(-130, -65)
    ax.set_ylim(24, 50)

    # We can now plot our GeoDataFrame.
    points_gdf.plot(ax=ax, color='blue')

    start_gdf.plot(ax=ax, color='green')
    stop_gdf.plot(ax=ax, color='red')

    points = []

    linedf = pd.DataFrame(
        {'Line': []})

    for i in range(len(points_gdf)):
        points.append(points_gdf.iloc[i].loc["geometry"])

    line = LineString(points)

    def plot_line(ax, ob):
        x, y = ob.xy
        ax.plot(x, y, color='grey', alpha=0.7, linewidth=1, solid_capstyle='round', zorder=2)

    plot_line(ax, line) # plot line of the show

    plt.tight_layout()

    plt.show()
    
# used to find the artist id on songkick
def get_artist_id(artist_name):
    json_str = requests.get("https://api.songkick.com/api/3.0/search/artists.json?apikey=" + api_key + "&query=" + artist_name).json()
    return (str(json_str['resultsPage']['results']['artist'][0]['id']) if json_str['resultsPage']['totalEntries'] != 0 else -1)         


# Analysis Functions

[Back to top](#Table-of-Contents)

In [4]:
# this function generates many related artists, returns most recent events for each one, 
def compile_points(names = [], followers = -1, event_num=8):
    if not (names == []):
            artists = []
            if followers == -1:   
                for n in names:
                    artists += related_artists(artname = n, llen = 10)
            else: # check for followers
                for n in names:
                    artists += related_artists(artname = n, llen = 10, followers = followers)

            nl = [] #list of names
            for a in artists:
                nl.append(a[0])
            
            nl = list(set(nl)) #only unique values
            
            gl = []
            
            for a in nl: # get points for each artist
                an = get_artist_id(a.replace(' ', '+')) if not (a[0]+' ').isspace() else print('need artist!')
    
                if an != -1:
                    req = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page='+str(event_num)+'&order=desc')
                    gdfs = make_points(req.json(), a)
                    if gdfs != -1:
                        gl.append(gdfs[0])
                
            return gl
    else:
        return -1


In [4]:
# create basic stats for top cities, states/countries, and venues
def basic_stats(gdfs, state = ''):    
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    
    # basic stats
    venues = {}
    cities = {}
    countries_states = {}
    
    for index, row in rdf.iterrows():
        if row[1] != 'Unknown venue':
            print(row)
            if row[1] not in venues:
                venues[row[1]] = 1
            else:
                venues[row[1]] += 1
          
        if row[3] != 'N/A':
            if (row[2] + ', ' + row[3][1:]) not in cities:
                cities[(row[2] + ', ' + row[3][1:])] = 1 # add city
            else:
                cities[(row[2] + ', ' + row[3][1:])] += 1 # increment
        else:
            if (row[2] + ', ' + row[4]) not in cities:
                cities[(row[2] + ', ' + row[4][1:])] = 1 # add city
            else:
                cities[(row[2] + ', ' + row[4][1:])] += 1 # increment
            
        if row[3] != 'N/A':
            if (row[3][1:]) not in countries_states:
                countries_states[row[3][1:]] = 1
            else:
                countries_states[row[3][1:]] += 1

        
        if (row[4][1:]) not in countries_states:
            countries_states[row[4][1:]] = 1
        else:
            countries_states[row[4][1:]] += 1
            
    print('Top 5 venues:')
    top = Counter(venues)
    for i in top.most_common(5):
        print(i[0] + ' ' + str(i[1]))
        
    print('\nTop 5 cities:')
    top = Counter(cities)
    for i in top.most_common(5):
        print(i[0] + ' ' + str(i[1]))
        
    print('\nTop 5 countries:')
    top = Counter(countries_states)
    for i in top.most_common(5):
        print(i[0] + ' ' + str(i[1]))
        
    return(venues)
        
    # end basic stats    

# conducts morans i test on data and plots
def autocorr(gdfs, state = ''):
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    plt.figure(figsize=(10, 12))

    world = ws
    
    showcount = [] # list that will be appened to the shapefile of states/countries
    
    # go through each country and find their amount of shows
    for index, row in world.iterrows():
        if row[1] == 'United States':
            c = 'US'            
        elif row[1] == 'United Kingdom':
            c = 'UK'
        else:
            c = row[1]
#             print(c)
        
        if c in countries_states:
            showcount.append(countries_states[c])
        else:
            showcount.append(0)
        
    world["ShowCount"] = showcount #make new column

    for index, row in ws.iterrows():
        if row[1] == 'United States':
            world.drop(index, inplace=True)
    
    fig, ax = plt.subplots(figsize=(15,15)) #plot
    world.plot("ShowCount", ax=ax)
    plt.tight_layout()
    plt.show()
    
    # morans i

    qW = Queen.from_dataframe(world)
    
    qW.silent_island_warning = True

    qW.transform = 'r'

    #moran
    moran_sc = esda.moran.Moran(world[['ShowCount']], qW)

    #lisa
    lisa_sc = esda.Moran_Local(world["ShowCount"], qW)

    print(str(moran_sc.I) + ", " + str(moran_sc.p_sim))
    
    from pysal.viz.splot import esda as esdaplot
    esdaplot.plot_local_autocorrelation(lisa_sc, world, "ShowCount")

    plt.show()    
    
def basic_analysis(gdfs, state = ''): # both functions together
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    basic_stats(gdfs)
    autocorr(gdfs)
    
# counts amount of shows by venue, for folium mapping and optimal tour path
def venue_count(gdfs, usonly = True):    
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    
    venues = {}
    
    if (usonly == True):
        for index, row in rdf.iterrows():
            if row[1] != 'Unknown venue':
                if (row[4].replace(' ', '') == 'US'): 
                    if row[1] not in venues.keys():
                        venues[row[1]] = [(row[5],row[6], row[2], row[3].replace(' ', '')), 1]
                    else:
                        venues[row[1]][1] += 1
        
    return(venues)

# gets tracts with certain demographic and socioeconomic data for venue locations
def get_tracts(vens):
    tracts = {}
    
    for k,v in vens.items():
        x = float(v[0][1])
        y = float(v[0][0])
        r = ''
        try:
            r = censusgeocode.coordinates(x=x, y=y) # finds tract
        except:
            print('censusgeocode error')
            continue
        
        try:
            r = r['2010 Census Blocks']
            st = r[0]['STATE']
            co = r[0]['COUNTY']
            tr = r[0]['TRACT']
            print(st + co + tr)
            
            #retrieving data from census
            b = cen.acs5.state_county_tract('B02001_002E,B02001_001E,B01002_001E,B19013_001E', st, co, tr)
            b2 = cen.acs5.state_county_tract('B02001_003E', st, co, tr)

            ftr = st + co + tr # full tract code
            if (ftr not in tracts.keys()):
                # factors of significance being calculated
                tracts[ftr] = {'Non-white population percentage': -1,
                'Black population percentage': -1,
                'Median age': -1,
                'Median household income': -1,
#                 'Nonfamily households percentage': -1,
                'Count': -1}
                # non-white/total pop
                tracts[ftr]['Non-white population percentage'] = (1 - b[0]['B02001_002E']/b[0]['B02001_001E'])
                # black/total
                tracts[ftr]['Black population percentage'] = b2[0]['B02001_003E']/b[0]['B02001_001E']
                tracts[ftr]['Median age'] = b[0]['B01002_001E']
                tracts[ftr]['Median household income'] = b[0]['B19013_001E']
#                 tracts[ftr]['Nonfamily households percentage'] = b2[0]['B11001_007E']/b2[0]['B11001_001E']
                tracts[ftr]['Count'] = v[1] # amount of shows in cenus tract
            else:
                tracts[ftr]['Count'] += v[1] # add to shows
        except KeyboardInterrupt:
            raise
        except:
            print('some other error')
            
    return tracts


In [6]:
# computes linear regression with tract information
def tract_reg(tr, demo='non-white'):
#     print(tr)

    # finding size of 2D list
    height = len(tr)
    width = len(next(iter(tr.values())))
    for k,vl in tr.items():
        v = list(vl.values())
        for i in range(width):
            if v[i] == -1:
#                 print(i)
                height = height - 1
                break

    print(height)
    print(width)

    # 2D list, each row is tract and columns are different values for census factors 
    arrr = [[0 for x in range(width-1)] for y in range(height)] 

    c = []

    i = 0
    
    # populating the 2D list with tracts and info, as long as no errors in its info
    for k,vl in tr.items():
        v = list(vl.values())

        con = False # if true, continue
        for j in range(width-1):
            if v[j] == -1:
                con = True

        if (con==True):
            continue

        c.append(v[width-1])
        for j in range(width-1):
            arrr[i][j] = v[j]
        i+=1
    
    # whether to use non-white or black percentage
    if (demo == 'black'):
        for row in arrr:
            del row[0]
    else:
        for row in arrr:
            del row[1]
        
    return statsmodels.regression.linear_model.OLS(c, arrr).fit().summary()


In [7]:
def tract_stats(names, followers, demo): # calls functions to do tract regression
    points = compile_points(names, followers = followers)
    tracts = get_tracts(venue_count(points))
    print(tract_reg(tracts, demo=demo))
    

In [13]:
# # from statsmodels.stats.outliers_influence import variance_inflation_factor
# # print(arrr)
# print(len(c))
# statsmodels.regression.linear_model.OLS(c, arrr).fit().summary()
# # print(statsmodels.)
# X = numpy.array(arrr)
# # print(X)
# vif = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
# print(vif)
# # statsmodels.regression.linear_model.OLS(c, arrr).fit().summary()


In [14]:
# req_input()

#travis scott, 2018-10-31, 2018-12-01

In [8]:
# this function generates many related artists, returns longest tours for each one, 
def compile_tours(names = [], followers = -1, lessthan = True):
    if (names != [] and isinstance(names, list)):
            artists = []
            
            # get 10 related artists for each name
            if followers == -1:   
                for n in names:
                    artists += related_artists(artname = n, llen = 10)
            else:
                for n in names:
                    artists += related_artists(artname = n, llen = 10, followers = followers)

            print(artists)
            nl = []
            for a in artists:
                nl.append(a[0])
            
            nl = list(set(nl)) # unique list of names
            
            print(len(nl))
            
            gl = []
            
            # for each artist, get show data and find longest tour
            for a in nl:
                an = get_artist_id(a.replace(' ', '+')) if not (a[0]+' ').isspace() else print('need artist!')
    
                if an != -1:
                    req = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page=50&order=desc')
#             print("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page=20&order=desc')
                    g = find_tour(req.json(), a) # find tour
                    if g is not int:
                        gl.append(g)
            
            return gl
    else:
        return -1


In [9]:
# just plots tours given multiple artists' tours
def plot_all_tours(ts):
    m = folium.Map(location=[40, -95], zoom_start=4)

    colors = ['blue', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen']

    cc = -1
    
    # plot each tour
    for x in ts:
        if not isinstance(x, int) and len(x) != 0:
            if (cc!=len(colors)-1):
                cc += 1
            else:
                cc = 0
    
            points = []

            for i, row in x.iterrows():
                tooltip = 'Artist: '+ str(row[0]) + '- City: ' + str(row[2]) + ', ' + str(row[3])

                points.append((row[5], row[6]))

                folium.Circle([row[5], row[6]], radius = 8, color = colors[cc], fill = True, weight = 5, popup='<i></i>', tooltip=tooltip).add_to(m)

            folium.PolyLine(points, color=colors[cc], weight=2.5, opacity=1).add_to(m)
            
    return m


In [10]:
# categorizes cities by the amount of shows they have, finds path of cities to tour
def optimal_path(ts):
    avg = 0
    maxm = 0
    cities = {}
    #     print(len(list))
    
    #tallying show counts by city
    for t in ts:
        if not isinstance(t, int):
            for i,row in t.iterrows():
                city = str(row[2]) + ', ' + str(row[3]) + ', ' + str(row[4])
                if city not in cities.keys():
                    cities[city] = [(row[5], row[6]), 1]
                else:
                    cities[city][1] += 1

    # sorting by show counts
    lst = sorted(cities.items(), key = 
             lambda kv:(kv[1][1], kv[0]), reverse=True)

    
    maxm = lst[0][1][1] #maximum shows
    sl = [] # list to find standard deviation
    for i in lst:
        avg += i[1][1]
        sl.append(i[1][1])

    sd = statistics.stdev(sl) # standard deviation

    avg = avg/len(lst) # average

    # lists for different classifications
    green = []
    lg = []
    yellow = []
    red = []

    # classifying
    for i in lst:
        count = i[1][1]
        if (count >= avg + sd*1.5):
            green.append(i[1][0])
        elif (count >= avg):
            lg.append(i[1][0])
        elif (count >= avg - sd/2):
            yellow.append(i[1][0])
        else:
            red.append(i[1][0])

    final_points = green # start with best points

    # distance between points
    def dist(x,y):
        return math.hypot(y[0]-x[0],y[1]-x[1])

    # creating path from point to point in final_points
    paths = [ p for p in it.permutations(final_points) ]
    path_distances = [ sum(map(lambda x: dist(x[0],x[1]),zip(p[:-1],p[1:]))) for p in paths ]
    min_index = numpy.argmin(path_distances)

    print(str(paths[min_index]) + ' ' + str(path_distances[min_index]))

    final_path = []

    # add created paths to final_path
    for i in paths[min_index]:
        final_path.append(i)

    print(final_path)

    # add to final_path if within a certain cutoff distance (std)
    def add_fp(fp, l, std):
        i = 0
        while (i < len(l)):
            if (l[i] not in fp):
                for j in range(len(fp)-1):
                    if ((dist(fp[j], l[i]) + dist(l[i], fp[j+1])) < (std*dist(fp[j], fp[j+1]))):
                        if (l[i] not in fp):
                            fp.insert(j+1, l[i])
                            print(l[i])
                            i = -1
                        continue
            i+=1

        return fp

    final_path = add_fp(final_path, lg, 1.5)
    final_path = add_fp(final_path, yellow, 1.2)
    final_path = add_fp(final_path, red, 1.05)

    print(final_path)

    m = folium.Map(location=[40, -95], zoom_start=4) # init folium map instance

    count = 0 # count of points to check
    for k, v in lst:
        color = ''
        if (v[0] in green):
            count+=1
            color = 'green'
        elif (v[0] in lg):
            count+=1
            color = 'lightgreen'
        elif (v[0] in yellow):
            count+=1
            color = 'yellow'
        elif (v[0] in red):
            count+=1
            color = 'red'
        else:
            print('oops')
                
        # add point to map
        tooltip = 'City: ' + str(k) + ' - Count: ' + str(v[1])
        folium.Circle([v[0][0], v[0][1]], radius = 8, color = color, fill = True, weight = 5, popup='<i></i>', tooltip=tooltip).add_to(m)

    # create line that represents final path from point to point
    folium.PolyLine(final_path, color='blue', weight=1, opacity=1.5).add_to(m)
  
    return m
    


In [9]:
# used to find the artist id on songkick
def get_artist_id(artist_name):
    json_str = requests.get("https://api.songkick.com/api/3.0/search/artists.json?apikey=" + api_key + "&query=" + artist_name).json()
    return (str(json_str['resultsPage']['results']['artist'][0]['id']) if json_str['resultsPage']['totalEntries'] != 0 else -1)

# function for interface for taking input
def req_input():
    print("Welcome")
    while (True):
        past = True
        is_search = True
        
        cmd = input("Search (0) or functions (1)")
        
        if cmd == 1:
            is_search = False
            
        if (is_search == False):
            cmd = input('''
            Basic analysis of the concerts for artist and their related artists? (0)\n
            Plot points for an artist, and their related artists? (1)\n
            Find optimal path for an artist? (2)\n
            Tract stats? (3)
            ''')
            
            if (cmd == 0):
                cmd = input("Type artist name: ")
                basic_analysis(cmd)
            elif (cmd == 1):
                cmd = input("Type artist name: ")
                plot_all_tours(compile_tours[cmd])
            elif (cmd == 2):
                cmd = input("Type artist name: ")
                optimal_path(compile_tours[cmd])
            elif (cmd == 3):
                cmd = input("Type artist name: ")
                tract_stats([cmd])
        else:
            cmd = input("Venue search (0) or event search (1)?")
        
            past = True
            is_venue = True
            if (cmd == 1):
                is_venue = False

            if (is_venue == False):
                cmd = input("Past events (0), upcoming events (1): ")

                if (cmd == 1):
                    past = False

                cmd = input("Please type the parameters: [artist_name, min_date, max_date.] Separate with commas."
                           + "\nIf no input, just add a comma and skip.\n")

                if (cmd == 'quit'):
                    return

                cs = cmd.split(',')
                an = get_artist_id(cs[0].replace(' ', '+')) if not (cs[0]+' ').isspace() else print('need artist!')
                if (cs[0]+' ').isspace():
                    continue
                mind = "&min_date=" + cs[1].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[1])) else ''
                maxd = "&max_date=" + cs[2].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[2])) else ''

                if (past == True):
                    r = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key 
                                  + mind + maxd)
                    run_plot(r, cs[0])
                else:
                    r = requests.get("https://api.songkick.com/api/3.0/events.json?apikey="+ api_key 
                        + an + loc + mind + maxd)
                    print(r.json())
                    run_plot(r, cs[0])
            else:
                cmd = input("Past venues (0), upcoming venues (1): ")

                if (cmd == 1):
                    past = False

            cmd = input("Please type the parameters: [artist_name, min_date, max_date.] Separate with commas."
                       + "\nIf no input, just add a comma and skip.\n")

            if (cmd == 'quit'):
                return

            cs = cmd.split(',')
            an = get_artist_id(cs[0].replace(' ', '+')) if not (cs[0]+' ').isspace() else print('need artist!')
            if (cs[0]+' ').isspace():
                continue
            mind = "&min_date=" + cs[1].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[1])) else ''
            maxd = "&max_date=" + cs[2].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[2])) else ''

            if (past == True):
                r = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key 
                                  + mind + maxd)
                run_plot(r, cs[0])
            else:
                r = requests.get("https://api.songkick.com/api/3.0/events.json?apikey="+ api_key 
                    + an + loc + mind + maxd)
                print(r.json())
                run_plot(r, cs[0])


# Analysis (look at other file for this section)

[Back to top](#Table-of-Contents)

Three genres: Hip-hop, Rock, Country and Four regions each: Midwest, East, West, South

Each pair of genre/region uses four or five artists that are from that area and under that genre.

These artists are used as the root artists to find related artists to get a sample of performance data.

Under each genre/region pair, there is:
* a regression done on the tracts the venues are found in
* plot of each artist's longest tour path from their past 50 shows
* a plot of the optimal path an artist would take with color coded points to signify the quality of each stop
    * Green: Shows > average + 1.5*sd
    * light green: Shows > average
    * Yellow: Shows > average - sd/2
    * red: everything else 

## Hip-hop

x1: (Black population)/(Total Population)

x2: Median Age

x3: Median Household Income

### Midwest (https://djbooth.net/features/2015-11-03-next-up-in-the-midwest)

In [922]:
tract_stats(names = ['pat app', 'supakaine', 'Kweku Collins', 'IshDARR'], followers = 50000, demo = 'black')

yo
related_artists search 1
related_artists ra_help 3
56
28
related_artists search 1
related_artists ra_help 3
110
78
related_artists search 1
related_artists ra_help 3
65
18
related_artists search 1
related_artists ra_help 3
67
23
25017353101
some other error
censusgeocode error
39049003200
29095015800
08031003701
censusgeocode error
censusgeocode error
censusgeocode error
censusgeocode error
53033008600
17031330100
censusgeocode error
censusgeocode error
censusgeocode error
37183052602
censusgeocode error
censusgeocode error
36047044900
13121000500
censusgeocode error
censusgeocode error
06037700501
48113020400
censusgeocode error
censusgeocode error
13121000500
censusgeocode error
17031061100
09003500100
48113020400
06073002201
censusgeocode error
censusgeocode error
06083000900
06037208502
censusgeocode error
censusgeocode error
53033010300
08031001701
04013218300
censusgeocode error
13121011900
censusgeocode error
censusgeocode error
censusgeocode error
48201310100
censusgeocode e

Rock:
    
    West: names = ['The Bonfire Set', 'Elephant Rifle', 'LO MOON', 'PINKY PINKY', 'My Goodness']
    East: names = ['Air Traffic Controller', 'Big Thief', 'J and The 9s', 'Charly Bliss', 'Hop Along']
    Midwest: names = ['Hemmingbirds', 'The Accidentals', 'The Cactus Blossoms', 'Chicken Wire Empire', 'Rodeo Ruby Love']
    South: names = ['Savoy Motel', 'Ringo Deathstarr', 'Gull', 'Shadowgraphs', 'Young Mister']



In [12]:
tours = compile_tours(names = ['The Bonfire Set', 'Elephant Rifle', 'LO MOON', 'PINKY PINKY', 'My Goodness'], followers = 50000)

# plot_all_tours(tours)

yo
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
128
72
related_artists search 1
related_artists ra_help 3
93
46
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
96
51
[('Big Black', '5EYkvHZuGM3pwU3DZUrrZ3', 14, 41236), ('The Jesus Lizard', '6r26MaDr8bqNALjXgYPXMa', 14, 46910), ('Wipers', '0sTTw3dw3EA0c7NaZnrJd2', 14, 28922), ('Unwound', '4YjpqCSDD7zwMQgPYJMqb0', 13, 28694), ('Mission Of Burma', '5UMQV83wwZCCvRSQCVjZw6', 13, 37760), ('Shellac', '6I8R5MFTlez7rHCsH4cx0u', 12, 48145), ('Flipper', '20VDXslLsXlLgSF765m1Ug', 9, 22700), ('Lightning Bolt', '2og3FOCLYXT9H7IYE6QPUq', 9, 44518), ('Brainiac', '0No2Og1pAwgw0eMmY6PPVP', 9, 8969), ('Scratch Acid', '219VCiRViG60EbPiocSWJp', 8, 16437), ('VHS Collection', '2Nvaq4y2ygxIqfwXyz0HeH', 7, 43655), ('Talos', '5pdzKTGQAcRcxDOfN4mXSc', 7, 38886), ('Mating Ritual', '4ZeB1hzT2mSZrf7wszOqHs', 6, 19305), ('courtship.', '2OK16hAFRHoJi

In [ ]:
optimal_path(tours)